# Partitioning

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
clean_9000 = 'result_clean_pdf_9000_files.csv'
clean_109 = 'result_clean_pdf_109_embed_video.csv'
malw_173 = 'result_malware_pdf_cve_sorted_173_files.csv'
malw_10982 = 'result_malware_pdf_pre_04-2011_10982_files.csv'
df_clean_9000 = pd.read_csv(clean_9000, sep=',')
df_clean_109 = pd.read_csv(clean_109, sep=',')
df_malw_173 = pd.read_csv(malw_173, sep=',')
df_malw_10982 = pd.read_csv(malw_10982, sep=',')

In [3]:
df_clean = pd.concat([df_clean_9000, df_clean_109], axis=0, ignore_index=True)
df_malw = pd.concat([df_malw_10982, df_malw_173], axis=0, ignore_index=True)

df_clean["malware"] = np.zeros(df_clean.shape[0], dtype='bool')
df_malw["malware"] = np.ones(df_malw.shape[0], dtype='bool')

In [4]:
df_tot = pd.concat([df_clean, df_malw], axis=0, ignore_index=True)

df_train, df_test = train_test_split(df_tot, test_size=0.1)
df_train, df_val = train_test_split(df_train, test_size=0.1)

df_train.to_csv("pdf_train.csv", index = False)
df_val.to_csv("pdf_val.csv", index = False)
df_test.to_csv("pdf_test.csv", index = False)